# Generate packer19 TreeData

Generate TreeData object using C.elegans data from [A lineage-resolved molecular atlas of C. elegans embryogenesis at single-cell resolution](https://www.science.org/doi/10.1126/science.aax1971)

## Setup

In [96]:
from pathlib import Path

import networkx as nx
import pandas as pd
import scipy as sp
import treedata as td

path = Path("/lab/solexa_weissman/wcolgan/pycea/datasets/packer19/")
data_path = path / "data"

## Load data

In [199]:
lineage = pd.read_excel(
    data_path / "aax1971_packer_tables_s1_to_s6_s9_s12_s13_s15_s16.xlsx", sheet_name="Table_S6", header=24
)
expression = pd.read_csv(data_path / "aax1971_Table_S8.gz", compression="gzip", sep="\t")
annotations = pd.read_excel(
    data_path / "aax1971_packer_tables_s1_to_s6_s9_s12_s13_s15_s16.xlsx", sheet_name="Table_S4", header=9
)
positions = pd.read_csv(data_path / "richards_positions.csv")

## Generate tree

In [206]:
lineage = lineage[["cell", "birth_time", "dies", "annotation_name", "clade", "generation_within_clade", "parent_cell"]]
tree = nx.DiGraph()
max_depth = 400
for _, row in lineage.iterrows():
    if row["birth_time"] >= max_depth:
        continue
    tree.add_node(row["cell"], **row.to_dict())
    if not pd.isna(row["parent_cell"]):
        if not pd.isna(row["clade"]):
            tree.add_edge(row["parent_cell"], row["cell"], clade=row["clade"])
        else:
            tree.add_edge(row["parent_cell"], row["cell"])
node_times = {}
for node in tree.nodes:
    children = list(tree.successors(node))
    if len(children) > 0:
        node_times[node] = min(tree.nodes[children[0]]["birth_time"], max_depth)
    elif tree.nodes[node]["dies"]:
        parent = list(tree.predecessors(node))
        node_times[node] = min(node_times[parent[0]] + 20, max_depth)
    else:
        node_times[node] = max_depth
nx.set_node_attributes(tree, node_times, "time")

## Generate TreeData

Scale positions to microns

In [200]:
positions["x"] = positions["x"] * 0.087
positions["y"] = positions["y"] * 0.087
positions["z"] = positions["z"] * 0.504
positions.drop_duplicates(subset=["cell"], inplace=True, keep="last")

Get counts, obs, and var

In [219]:
var = expression.groupby("gene").agg({"gene.id": "first"}).rename(columns={"gene.id": "gene_id"})
counts = expression.pivot_table(index="lineage", columns="gene", values="adjusted.tpm.estimate")
obs = lineage.query("cell.isin(@tree.nodes) & annotation_name.isin(@counts.index)")
obs = obs.merge(
    annotations[["Lineage", "Lineage group", "UMAP", "Cells produced"]].rename(
        columns={
            "Lineage": "annotation_name",
            "Lineage group": "lineage_group",
            "UMAP": "umap_cluster",
            "Cells produced": "cells_produced",
        }
    ),
    on="annotation_name",
    how="left",
)
obs = obs.merge(positions[["cell", "x", "y", "z"]], on="cell", how="left").set_index("cell")
obs["time"] = obs.index.map(node_times)
tdata = td.TreeData(
    X=sp.sparse.csr_matrix(counts.loc[obs.annotation_name]), obs=obs, var=var, obst={"tree": tree}, alignment="subset"
)
tdata.obsm["spatial"] = obs[["x", "y", "z"]].values

## Save TreeData

In [223]:
tdata.write_h5td(path / "packer19.h5td")